In [46]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
dictionary_words = set(nltk.corpus.words.words())

from pprint import pprint

from textblob import TextBlob, Word

import spacy
import spacy_legacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words


import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models

import itertools 
#custom list of stop words
stop_words_english = []
with open('stop_words_english.txt', encoding="utf8") as my_file:
    for line in my_file:
        stop_words_english.append(line.replace("\n", ""))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
df = pd.read_csv ('AllReviews0120-1122.csv')

In [48]:
df.shape

(35895, 3)

In [49]:
df

,Page Type,Full Text,Rating
0,review,"When they work they are great, however, in my ...",1
1,review,I have a garmin Vivosmart 4 and this fitbit ch...,1
2,review,Absolute Trash! Seriously felt like I bought t...,1
3,review,I got this to pair with my apple watch and cha...,1
4,review,I am very disappointed with the FitBit Charge ...,1
...,...,...,...
35890,review,Love it! I have a very small wrist so the fact...,5
35891,review,I loved this ! Better than my Apple Watch but ...,5
35892,review,Apple Watch Series 3 (GPS) 38mm Silver Aluminu...,5
35893,review,I love my Apple Watch. Helps me keep track of ...,5


In [50]:
df= df.apply(lambda x: x.astype(str).str.lower())

In [51]:
df.head()

,Page Type,Full Text,Rating
0,review,"when they work they are great, however, in my ...",1
1,review,i have a garmin vivosmart 4 and this fitbit ch...,1
2,review,absolute trash! seriously felt like i bought t...,1
3,review,i got this to pair with my apple watch and cha...,1
4,review,i am very disappointed with the fitbit charge ...,1


In [52]:
reviews = np.array(df['Full Text'])
reviews = ' '.join(reviews)
#reviews

# Identify Keywords with Negators and Remove Review

In [53]:
from re import split
def split_string(string, delimiters):
    string = string.replace(".", "," ) 
    string = string.replace("?", "," )
    pattern = r'|'.join(delimiters)
    return split(pattern, string)

In [54]:
r = "Python is fun. The string is for test because python run the code once you execute it"

#conjunctions and punctuations are based on the Buschken paper, excpet the '(' and ')'
d = ['& ', ', ','; ','! ',': ','for ','and ', 'but ', 'or ', 'so ',
     'after ', 'as ', 'because ', 'before ', 'even ', 'if ', 'now ', 'once ', 
     'since ', 'than ', 'that ', 'thought ', 'when ', 'where ', 'which ', 
     'while ', 'who ', 'what ']

new_string = split_string(r, d)
print(new_string)

['Python is fun', 'The string is ', 'test ', 'python run the code ', 'you execute it']


In [55]:
# #custom list of stop words without conjunctions
# stop_words_english = []
# with open('stop_words_english_conjunctions_negations.txt', encoding="utf8") as my_file:
#     for line in my_file:
#         stop_words_english.append(line.replace("\n", ""))

In [56]:
def clean_text(docs):
    for i in range(len(docs)):
        #remove hashtags and mentions
        docs[i] = " ".join(filter(lambda x:x[0]!='#', docs[i].split())) #remove hashtags
        docs[i] = " ".join(filter(lambda x:x[0]!='@', docs[i].split())) #remove mentiones
        
        #remove special chars, anything not a number or letter, anything excpet alphanumeric words
        docs[i] = re.sub(r'[^\w\s]','',docs[i]) #remove punctuation marks and special chars
        docs[i] = re.sub('[^A-Za-z0-9]+', ' ', docs[i]) #remove anything that is not a number or or letter
        docs[i] = re.sub('[\W_]', ' ', docs[i]) #Remove any thing excpet charcater and alphanumeric words
        docs[i] = re.sub(r'\w*\d\w*', '', docs[i]) #Remove any thing with numbers
        docs[i] = ' '.join(word for word in docs[i].split() if not word.isdigit())
        
        #compare words to englisg dictionary and remove nonsense words
        words = [word for word in docs[i].split() if word.lower() in dictionary_words or not word.isalpha()]
        docs[i] = " ".join(words)
        
        #remove words that consists of less than three letter
        words = [word for word in docs[i].split() if len(word)>2]
        docs[i] = " ".join(words)
                             
        #Remove stop words using custom list of stop words
        #words = [word for word in reviews[i].split() if word.lower() not in stop_words_english]
        #reviews[i] = " ".join(words)

        #Remove brands related keywords
        brands = ["fitbit ace", "fitbit charge", "fitbit verca", "fitbit ionic",  "fitbit versa lite",  
                  "fitbit inspire",  "fitbit versa",  "garmin forerunner",  "garmin fēnix",  "garmin vívoki",  
                  "garmin vívofit",  "garmin vívoactive",  "garmin vívomove",  "garmin vívosmart",  
                  "garmin vívosport",  "garmin venu",  "garmin instinct",  "garmin luxe",  "garmin darth vader",  
                  "garmin captain marvel",  "garmin approach",  "garmin lily",  "garmin swim",  "xiaomi mi band",  
                  "xiaomi mi samrt band",  "moov now",  "moov hr",  "apple watch",  "fossil gen",  
                  "fossil sport smartwatch",  "fossil hybrid smartwatch hr",  "misfit vap, ",  "misfit ray",  
                  "misfit path",  "misfit command",  "withings steel",  "withings move",  "withings pulds",  
                  "withings move ecg",  "ihealth watch",  "samsung gear",  "samsung galaxy watch active",  
                  "samsung galaxy fit",  "polar a360",  "polar a370",  "polar m430",  "polar m200",  "polar h10",  
                  "polar vantage",  "polar ignite",  "polar grit x",  "polar titan",  "polar oh1",  "polar h9",  
                  "striiv fusion",  "striiv apex hr",  "striiv dash hr",  "huawei talkband",  " huawei watch",  
                  "huawei band",  "mykronoz zewatch",  "mykronoz zefit",  "mykronoz zesport",  "mykronoz zetime",  
                  "mykronoz zefit",  "mykronoz zeround",  "mykronoz zeneo",  "mykronoz zetrack",  "coros apex",  
                  "wyze band",  "letsfit fitness racker",  "coros pace",  "withings scanwatch",  "amazon halo",  
                  "timex ironman",  "suunto peak",  "wahoo fitness", "fitbit", "garmin", "apple", "xiaomi", "moov",
                  "fossil", "misfit", "withings", "ihealth", "samsung", "polar", "striiv", "huawei", "mykronoz", 
                  "coros", "wyze", "letsfit", "amazon"]

        resultwords  = [word for word in reviews[i].split() if word.lower() not in brands]
        reviews[i] = ' '.join(resultwords)


        #Remove category-related words
        categories = ["watch", "watches", "smart watch", "smartwatch", "band", "bands", "smartband", 
                      "smart band", "smartbands", "smart bands", "case", "cases", "cover", "protective", 
                      "protector", "screen", "show", "iphone", "express", "appleevent", "shield", 
                      "show", "appltv", "gear vr", "tv", "shoe", "mountain", "mountains", "beach", 
                      "appltv", "accessory", "accessories"]

        resultwords  = [word for word in reviews[i].split() if word.lower() not in categories]
        reviews[i] = ' '.join(resultwords)


        #Remove feature-related words
        features = ["ability", "quality", "feature", "aspect", "abilities", "qualities", "features", "aspects"]
        resultwords  = [word for word in reviews[i].split() if word.lower() not in features]
        reviews[i] = ' '.join(resultwords)

    return docs

In [57]:
reviews = np.array(df['Full Text'])
processed_reviews = clean_text(reviews)

In [58]:
processed_reviews

array(['when they work they are great however one year warranty had the device twice this most recent time just outside warranty rather than replace the device for third failure they off code for new device from their shop where the are higher than most retail decided much like the and interface going have control just too poor',
       'have and this charge however the not accurate all wore and the same time when running also carried phone see which one more accurate the same distance phone did but was about less than the moreover cannot convert your activity the',
       'absolute trash seriously felt like bought this through candy machine was nightmare was not consistent with when cycling would switch just waste time wasnt consistent sent back and inspire and much happier',
       ..., 'series silver aluminum with white sport silver aluminum',
       'love keep track fitness and workout more than would normally',
       'really the helping track nice wide display'], dtype=object)

In [59]:
d = ['& ', ', ','; ','! ',': ','for ','and ', 'but ', 'or ', 'so ',
     'after ', 'as ', 'because ', 'before ', 'even ', 'if ', 'now ', 'once ', 
     'since ', 'than ', 'that ', 'thought ', 'when ', 'where ', 'which ', 
     'while ', 'who ', 'what ']

for i in range(len(processed_reviews)):
    processed_reviews[i] = split_string(processed_reviews[i], d)

processed_reviews

array([list(['', 'they work they are great however one year warranty had the device twice this most recent time just outside warranty rather ', 'replace the device ', 'third failure they off code ', 'new device from their shop ', 'the are higher ', 'most retail decided much like the ', 'interface going have control just too poor']),
       list(['have ', 'this charge however the not accurate all wore ', 'the same time ', 'running al', 'carried phone see ', 'one more accurate the same distance phone did ', 'w', 'about less ', 'the moreover cannot convert your activity the']),
       list(['absolute trash seriously felt like bought this through candy machine w', 'nightmare w', 'not consistent with ', 'cycling would switch just waste time wasnt consistent sent back ', 'inspire ', 'much happier']),
       ...,
       list(['series silver aluminum with white sport silver aluminum']),
       list(['love keep track fitness ', 'workout more ', 'would normally']),
       list(['really the helpi

In [60]:
print(type(processed_reviews))

<class 'numpy.ndarray'>


In [61]:
df1 = pd.DataFrame(processed_reviews)
df1.columns = ['Full Text']
df1.head()

,Full Text
0,"[, they work they are great however one year w..."
1,"[have , this charge however the not accurate a..."
2,[absolute trash seriously felt like bought thi...
3,"[got this pair with , chart easier this rated ..."
4,[very disappointed with the charge had chosen ...


In [62]:
df1 = df1.join(df['Rating'])

In [63]:
df1

,Full Text,Rating
0,"[, they work they are great however one year w...",1
1,"[have , this charge however the not accurate a...",1
2,[absolute trash seriously felt like bought thi...,1
3,"[got this pair with , chart easier this rated ...",1
4,[very disappointed with the charge had chosen ...,1
...,...,...
35890,[love have very small wrist the factory made w...,5
35891,"[this better , , only about main blank can cli...",5
35892,[series silver aluminum with white sport silve...,5
35893,"[love keep track fitness , workout more , woul...",5


In [64]:
############## Check Terms ####################

#terms = ['easy to use', 'simple', 'straightforward', 'user-friendly', 'intuitive']

def checkterms(string, termsvalues):
    checkterm = False
    for i in range(len(termsvalues)):
        if (string.find(termsvalues[i]) >= 0):
            checkterm = True
    return checkterm
 
#checkterms('This is easy to use hello', terms)

In [65]:
############## Check Negators ####################
def checknegators(string):
    negatorvalues = ["ain't", "aren't", "cannot", "cant", "can't", "couldn't", "daren't", "didn't", 
                     "don't", "hasn't","haven't", "isn't", "mayn't","mightn't","mustn't","needn't",
                     "not","oughtn't","shan't","shouldn't","weren't", "won't", "wouldn't"]
    checknegator = False
    for i in range(len(negatorvalues)):
        if (string.find(negatorvalues[i]) >= 0):
            checknegator = True
    return checknegator
 
#checknegators('This is hello simple')

In [66]:
############## Encode Reviews####################


In [67]:
#myArray = ['easy to use watch', 'health ', 'attention']

#ease_of_use_terms = ['simple', 'easy to use',  'straightforward', 'user-friendly', 'intuitive']


def encodereviews(reviewarray, qualityterms):
    encode = 0
    for i in reviewarray:
        if((checknegators(i) == False) and (checkterms(i, qualityterms) == True)):
            encode = 1
        else:
            continue
    return encode    

In [68]:
#encodereviews(myArray, ease_of_use_terms)

In [69]:
df1["Hedonic Motivation"] = ""

terms = ['enjoyable', 'enjoyment', 'enjoy', 'pleasant', 'flow experience', 'pleasure', 'fun', 
         'interesting', 'satisfied', 'satisfaction', 'feel good', 'happiness', 'happy', 'happier', 
         'entertainment', 'entertain', 'exciting', 'playfulness', 'playful',  'feeling of accomplishment']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Hedonic Motivation'] = 1
    else:
        df1.at[i,'Hedonic Motivation'] = 0
        

df1.head()

,Full Text,Rating,Hedonic Motivation
0,"[, they work they are great however one year w...",1,0
1,"[have , this charge however the not accurate a...",1,0
2,[absolute trash seriously felt like bought thi...,1,1
3,"[got this pair with , chart easier this rated ...",1,0
4,[very disappointed with the charge had chosen ...,1,0


In [70]:
df1["Connectivity Support"] = ""

terms = ['pair', 'sync', 'connect']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Connectivity Support'] = 1
    else:
        df1.at[i,'Connectivity Support'] = 0
    
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support
0,"[, they work they are great however one year w...",1,0,0
1,"[have , this charge however the not accurate a...",1,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0
3,"[got this pair with , chart easier this rated ...",1,0,1
4,[very disappointed with the charge had chosen ...,1,0,0


In [71]:
df1["Customizability"] = ""

terms = ['customize', 'custom', 'personalize', 'personalization']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Customizability'] = 1
    else:
        df1.at[i,'Customizability'] = 0
        
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability
0,"[, they work they are great however one year w...",1,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0
4,[very disappointed with the charge had chosen ...,1,0,0,1


In [72]:
df1["Perceived Ease of Use"] = ""

terms = ['easy', 'clear', 'understandable', 'easy to use', 'mental effort', 
         'trouble free', 'simple', 'controllable', 'easy to learn']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Perceived Ease of Use'] = 1
    else:
        df1.at[i,'Perceived Ease of Use'] = 0
        
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use
0,"[, they work they are great however one year w...",1,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0


In [73]:
df1["Appeal"] = ""

terms = ['cool', 'appearance', 'aesthetic', 'stylish', 'fashion', 'accessory', 'pleasing aesthetics', 'color', 
         'design', 'texture', 'uniqueness', 'unique', 'size', 'look']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Appeal'] = 1
    else:
        df1.at[i,'Appeal'] = 0
        
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal
0,"[, they work they are great however one year w...",1,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0


In [74]:
df1["Perceived Value"] = ""

terms = ['price', 'value', 'cost', 'monetary', 'money']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Perceived Value'] = 1
    else:
        df1.at[i,'Perceived Value'] = 0

df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0


In [75]:
df1["Perceived Usefulness"] = ""

terms = ['improve', 'enhance', 'useful', 'accomplish', 'increase', 'rejuvenated', 'improve', 'simplify', 
         'control over', 'reduce', 'accomplish more', 'save time', 'work more quickly']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Perceived Usefulness'] = 1
    else:
        df1.at[i,'Perceived Usefulness'] = 0
        
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0


In [76]:
df1["Device Quality"] = ""

terms = ['quality', 'system quality', 'service quality', 'battery life', 'charge', 'long time', 'battery last']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Device Quality'] = 1
    else:
        df1.at[i,'Device Quality'] = 0
        
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0,0
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0,1


In [77]:
df1["Credibility Support"] = ""

terms = ['accuracy', 'accurate']

for i in range(len(df1['Full Text'])):
    #print(df1['Full Text'][i])
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Credibility Support'] = 1
    else:
        df1.at[i,'Credibility Support'] = 0
        
df1.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0,0,1
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0,1,0


In [78]:
df2 = df1
df2.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0,0,1
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0,0,0
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0,0,0
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0,1,0


In [80]:
df2["Sum"] = ""
df2.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support,Sum
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0,0,0,0,
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0,0,1,
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0,0,0,
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0,0,0,
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0,1,0,


In [81]:
for i in range(len(df2['Full Text'])):
    df2['Sum'][i] = df2['Perceived Ease of Use'][i] + df2['Hedonic Motivation'][i] + \
                df2['Connectivity Support'][i] + df2['Customizability'][i] + df2['Appeal'][i] + \
                df2['Perceived Value'][i] + df2['Perceived Value'][i] + df2['Perceived Usefulness'][i] + \
                df2['Device Quality'][i] + df2['Credibility Support'][i]

In [82]:
df2.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support,Sum
0,"[, they work they are great however one year w...",1,0,0,0,0,0,0,0,0,0,0
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0,0,1,1
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0,0,0,1
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0,0,0,1
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0,1,0,2


In [83]:
df2 = df2[df2.Sum != 0]

In [84]:
df2.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support,Sum
1,"[have , this charge however the not accurate a...",1,0,0,0,0,0,0,0,0,1,1
2,[absolute trash seriously felt like bought thi...,1,1,0,0,0,0,0,0,0,0,1
3,"[got this pair with , chart easier this rated ...",1,0,1,0,0,0,0,0,0,0,1
4,[very disappointed with the charge had chosen ...,1,0,0,1,0,0,0,0,1,0,2
6,"[, third piece crap quits with the br, neither...",1,1,0,0,0,0,0,0,0,0,1


In [85]:
df2.to_csv('ProcessedReviews.csv')